In [1]:
import cv2
import numpy as np
des_pts = np.array([[200,100], [800,100], [200,700], [800,700]])
des_cm = np.array([[200,100], [383,100], [200,283], [383,283]])
src_pts = np.array([[516,22], [971,182], [206,392], [730,685]])

h, status = cv2.findHomography(src_pts, des_cm)
np.save('h_cm.npy', h, False)

In [1]:
import cv2, time

vid = cv2.VideoCapture(0)
vid.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
time.sleep(2)

_, img = vid.read()
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [19]:
warp_img = cv2.warpPerspective(img, h, (img.shape[1],img.shape[0]))

In [5]:
h

array([[ 4.79145899e-01, -5.40578721e-01,  4.34839724e+02],
       [ 2.04831124e-01,  4.49040132e-01, -6.55496126e+01],
       [-2.27909269e-04, -3.07533559e-04,  1.00000000e+00]])

In [20]:
cv2.imshow('img', warp_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
import cv2, time
import numpy as np
vid = cv2.VideoCapture(0)
vid.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
time.sleep(2)
h = np.load('h_cm.npy')
while True:
    _, img = vid.read()
    img = cv2.circle(img, (500, 500), 1, color=(0,0,255), thickness=9)
    warp_img = cv2.warpPerspective(img, h, (img.shape[1],img.shape[0]))
    cv2.imshow('img', warp_img)
   #  cv2.imwrite('template.jpg', img)
   #  if True:
   #     break
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
import cv2, time
import mediapipe as mp
import numpy as np
import time


mp_objectron = mp.solutions.objectron
vid = cv2.VideoCapture(0)
vid.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
time.sleep(2)


iterations = [1, 2, 5, 6]
h = np.load('h.npy')
temp_img = cv2.imread('template.jpg')
while True:
    start_time = time.time()
    _, img = vid.read()
    with mp_objectron.Objectron(static_image_mode=True, max_num_objects=6, min_detection_confidence=0.5, model_name='Shoe') as objectron:
   
      results = objectron.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

      if not results.detected_objects:
        print('No box landmarks detected on shoe')
        continue
      #print('Box landmarks of shoe:')
      ann_img = img.copy()
      temp_img_copy = temp_img.copy()
      display_img = temp_img_copy
      for detected_object in results.detected_objects:
        for i in iterations:
          x = int(detected_object.landmarks_2d.landmark[i].x * img.shape[1])
          y = int(detected_object.landmarks_2d.landmark[i].y * img.shape[0])
          display_img = cv2.circle(display_img, (x,y), 1, color=(0, 0, 255), thickness=5)
      if cv2.waitKey(1) & 0xFF == ord('q'):
        break
      warp_img = cv2.warpPerspective(display_img, h, (img.shape[1],img.shape[0]))
      cv2.imshow('img', warp_img)
    end_time = time.time()
    print(end_time - start_time)
cv2.waitKey(0)
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmarks detected on shoe
No box landmar

: 

: 

In [58]:
import cv2
import numpy as np
import time

import mediapipe as mp

#img = cv2.imread('shoe.png')
#img = cv2.imread('/home/hameed/Downloads/shoes.jpeg')
vid = cv2.VideoCapture(0)
# vid.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
# vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
time.sleep(2)

def midpoint(x1, y1, x2, y2):
    return ((x1 + x2)/2, (y1 + y2)/2)

def point_transform(p, h):
    p = np.array(p, dtype='float32').reshape((1,1,2))
    p_out = cv2.perspectiveTransform(p, h)[0,0,:]
    return p_out


mp_objectron = mp.solutions.objectron

fx, fy = (1.0, 1.0)
px, py = (0.0, 0.0)
iterations = [1, 2, 5, 6]



axis_world = np.float32([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])
while True:
    _, img = vid.read()
    objectron = mp_objectron.Objectron(mp_objectron.Objectron(static_image_mode=True, max_num_objects=5, min_detection_confidence=0.5, model_name='Shoe'))
    results = objectron.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ann_img = img.copy()
    #warp_img = img.copy()
    x = []
    y = []
    z_axis = []
    if not results.detected_objects:
        print('no detected objects', end='\r')
        continue
    for obj in results.detected_objects:
        
        x.append(int(obj.landmarks_2d.landmark[0].x * img.shape[1]))
        y.append(int(obj.landmarks_2d.landmark[0].y * img.shape[0]))

        axis_cam = np.matmul(obj.rotation, 0.1 * axis_world.T).T + obj.translation
        x_ori = axis_cam[..., 0]
        y_ori = axis_cam[..., 1]
        z_ori = axis_cam[..., 2]
    # Project 3D points to NDC space.
        x_ndc = np.clip(-fx * x_ori / (z_ori + 1e-5) + px, -1., 1.)
        y_ndc = np.clip(-fy * y_ori / (z_ori + 1e-5) + py, -1., 1.)
    # Convert from NDC space to image space.
        x_im = np.int32((1 + x_ndc) * 0.5 * img.shape[1])
        y_im = np.int32((1 - y_ndc) * 0.5 * img.shape[0])
        z_axis.append((x_im[3], y_im[3]))

    try:
        x, y = midpoint(x[0], y[0], x[1], y[1])
        x, y = point_transform((x,y), h)
        z0, z1 = midpoint(z_axis[0][0], z_axis[0][1], z_axis[1][0], z_axis[1][1])
        z0, z1 = point_transform((z0, z1), h)

        warp_img = cv2.warpPerspective(ann_img, h, (img.shape[1],img.shape[0]))
        warp_img = cv2.circle(warp_img, (int(x), int(y)), radius=100, color=(0, 0, 255), thickness=5)
        warp_img = cv2.arrowedLine(warp_img, (int(x), int(y)), (int(z0), int(z1)), color=(0, 128, 0), thickness=3, tipLength=0.5)
    except:
        print('not enough points', end='\r')

    try:
        cv2.imshow('ann_img', warp_img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    except:
        print('wait', end='\r')
cv2.waitKey(0)
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


: 

: 

In [8]:
if results.detected_objects:
    print(1)
else:
    print(0)

0


In [4]:
import time

i = 0
while i < 5:
    print(i, end='\r')
    time.sleep(2)
    i += 1

In [56]:
import cv2
import numpy as np

p1 = np.array((400, 400), dtype='float32').reshape((1,1,2))
#p1 = np.array([p1])
h = np.load('h.npy')
p1_t = cv2.perspectiveTransform(p1, h)
tuple(p1_t[0,0,:].shape)


(2,)

In [48]:
def point_transform(p, h):
    p = np.array(p, dtype='float32').reshape((1,1,2))
    p_out = cv2.perspectiveTransform(p, h)[0,0,:]
    return p_out

In [57]:
a, b = point_transform((400,400), h)
a, b


(387.8386, 613.14435)